In [ ]:
# Import Stuff
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from multiprocessing import Process
from IPython.display import clear_output

In [ ]:
!pip  install imgextract

In [ ]:
# All Parameters required for training are declared over here
# Frequency of Image Capturing
FRAME_SKIP = 2
# Frame Size
FRAME_SIZE = (150,150)

In [ ]:
import subprocess

# Remove the existing directory if it exists
subprocess.run(['rm', '-rf', 'A-Dataset-for-Automatic-Violence-Detection-in-Videos/'])

# Clone the repository
subprocess.run(['git', 'clone', 'https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos'])


CompletedProcess(args=['git', 'clone', 'https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos'], returncode=0)

In [ ]:
!rm -r Data
!mkdir Data
!mkdir -p ./Data/Video/Violent
!mkdir -p ./Data/Video/NonViolent
!cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/violent/cam1/. ./Data/Video/Violent/
!cp -a ./A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/non-violent/cam1/. ./Data/Video/NonViolent/
from IPython.display import clear_output
clear_output()
!mkdir -p ./Data/Training/V
!mkdir -p ./Data/Training/NV

In [ ]:
import cv2
import os

class Extractor:
    def __init__(self, frame_size, frame_skip):
        self.frame_size = frame_size
        self.frame_skip = frame_skip

    def extract(self, video_path, category):
        # Your frame extraction logic goes here
        cap = cv2.VideoCapture(video_path)
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Your processing logic on each frame goes here
            # For example, you can save the frame as an image file
            frame_count += 1
            frame_path = f"./Data/Training/{category}/frame_{frame_count}.jpg"
            cv2.imwrite(frame_path, frame)

            # Skip frames based on the specified frame_skip
            for _ in range(self.frame_skip - 1):
                cap.read()

        cap.release()





In [ ]:
def thread_1():
    ext = Extractor(FRAME_SIZE, FRAME_SKIP)
    for i in range(60):
        path = f"./Data/Video/Violent/{i+1}.mp4"
        print(f"Processing Violent Vid-{i}")
        ext.extract(path, 'V')
    print("Violent Extracted")

def thread_2():
    ext = Extractor(FRAME_SIZE, FRAME_SKIP)
    for i in range(60):
        path = f"/content/Data/Video/NonViolent/{i+1}.mp4"
        print(f"Processing NonViolent Vid-{i}")
        ext.extract(path, 'NV')
    print("Non-Violent Extracted")

In [ ]:
# Violent Extraction
from multiprocessing import Process

# Your other imports and code here

t1 = Process(target=thread_1, args=())
t2 = Process(target=thread_2, args=())

t1.start()
t2.start()
# NonViolent Extraction

t1.join()
t2.join()
print("Complete")

Processing Violent Vid-0
Processing NonViolent Vid-0
Processing Violent Vid-1
Processing NonViolent Vid-1
Processing NonViolent Vid-2
Processing Violent Vid-2
Processing NonViolent Vid-3
Processing Violent Vid-3
Processing NonViolent Vid-4
Processing Violent Vid-4
Processing Violent Vid-5
Processing NonViolent Vid-5
Processing NonViolent Vid-6
Processing Violent Vid-6
Processing NonViolent Vid-7
Processing Violent Vid-7
Processing NonViolent Vid-8
Processing Violent Vid-8
Processing NonViolent Vid-9
Processing Violent Vid-9
Processing NonViolent Vid-10
Processing NonViolent Vid-11
Processing Violent Vid-10
Processing NonViolent Vid-12
Processing Violent Vid-11
Processing Violent Vid-12
Processing NonViolent Vid-13
Processing Violent Vid-13
Processing NonViolent Vid-14
Processing Violent Vid-14
Processing Violent Vid-15
Processing NonViolent Vid-15
Processing NonViolent Vid-16
Processing Violent Vid-16
Processing Violent Vid-17
Processing NonViolent Vid-17
Processing Violent Vid-18
Proc

In [ ]:
base_dir='./Data'
train_dir=os.path.join(base_dir,'Training')
train_violent_dir =os.path.join(train_dir, 'V' )
train_nonviolent_dir=os.path.join(train_dir,'NV')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.25)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    color_mode="rgb",
    target_size=FRAME_SIZE,
    batch_size=20,
    classes=['NV', 'V'],
    class_mode='binary',
    shuffle=True,
    subset='training'  # This will select the training subset
)

Found 302 images belonging to 2 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    color_mode="rgb",
    target_size=FRAME_SIZE,
    batch_size=20,
    classes=['NV', 'V'],
    class_mode='binary',
    shuffle=True,
    subset='validation'  # This will select the validation subset
)


Found 100 images belonging to 2 classes.


In [ ]:
import math
steps_per_epoch = math.ceil(train_generator.samples / train_generator.batch_size)

In [ ]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch ,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/5
16/16 [==============================] - ETA: 0s - loss: 0.7373 - accuracy: 0.4834

16/16 [==============================] - 10s 504ms/step - loss: 0.7373 - accuracy: 0.4834 - val_loss: 0.6889 - val_accuracy: 0.6700
Epoch 2/5
16/16 [==============================] - 5s 336ms/step - loss: 0.6882 - accuracy: 0.5497
Epoch 3/5
16/16 [==============================] - 5s 322ms/step - loss: 0.6314 - accuracy: 0.6391
Epoch 4/5
16/16 [==============================] - 5s 333ms/step - loss: 0.5705 - accuracy: 0.6854
Epoch 5/5
16/16 [==============================] - 6s 393ms/step - loss: 0.3693 - accuracy: 0.8311


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# Plot training & validation accuracy values
history.plot(metric='accuracy')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
history.plot(metric='loss')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print("Testing Accuracy:", test_accuracy)

In [ ]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

In [ ]:
import time

# Define your desired model name here
model_name = "CNN"

# Define the export path with the custom model name
export_path_keras = f"./{model_name}.h5"

# Save the model with the specified export path
model.save(export_path_keras)

print("Model saved as:", export_path_keras)
